In [7]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json

def extract_reviews_from_page(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    data = soup.find_all("script", type="application/ld+json")
    reviews = []
    for contents in data:
        jsonData = json.loads(contents.text)
        if "review" in jsonData:
            for review in jsonData["review"]:
                headline = review.get("headline", "")
                review_body = review.get("reviewBody", "")
                rating_value = review["reviewRating"]["ratingValue"]
                reviews.append({"MainReview": headline, "Detailed_Review": review_body, "Rating": rating_value})
    return reviews

def scrape_reviews(url, pages):
    browser = webdriver.Chrome()
    results = []
    for page_num in range(1, pages+1):
        current_url = f"{url}?bvstate=pg:{page_num}/ct:r#ratings_section"
        browser.get(current_url)
        time.sleep(5)  # Wait for the new content to load
        html = browser.page_source
        reviews = extract_reviews_from_page(html)
        results.extend(reviews)
    browser.quit()
    return results

if __name__ == "__main__":
    url = "https://www.dell.com/en-in/work/shop/business-laptop-notebook-computers/latitude-7440-laptop/spd/latitude-14-7440-2-in-1-laptop/n001l7440017in9_vp"
    total_pages = 20
    reviews = scrape_reviews(url, total_pages)
    df = pd.DataFrame(reviews)
    df.to_csv('reviewsfinal123.csv', index=False)
